https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24

In [1]:
# import gensim
# from gensim.utils import simple_preprocess
# from gensim.parsing.preprocessing import STOPWORDS
# from nltk.stem import WordNetLemmatizer, SnowballStemmer
# from nltk.stem.porter import *
# import numpy as np
# import nltk

# np.random.seed(2018)
# nltk.download('wordnet')

In [2]:
# import logging
# from time import time
# np.random.seed(2018)

# #logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# #lda_model = gensim.models.LdaModel(bow_corpus, num_topics=10, id2word=dictionary,  eval_every=5, iterations = 1000, alpha='auto', gamma_threshold=0.01)
# t0 = time()
# #lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, iterations = 1000, passes = 2)
# lda_model = gensim.models.LdaModel(bow_corpus, num_topics=10, id2word=dictionary,  eval_every=5, iterations = 1000, alpha='auto', gamma_threshold=0.01)
# print("done in %fs" % (time() - t0))

# for idx, topic in lda_model.print_topics(-1):
#     print('Topic: {} \nWords: {}'.format(idx, topic))

In [3]:
from gensim import corpora, models
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
import numpy as np
from tqdm import tqdm
from time import time
import json

np.random.seed(2018)

stemmer = SnowballStemmer('english')

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in simple_preprocess(text, min_len = 4):
        if token not in STOPWORDS:
            result.append(lemmatize_stemming(token))
    return result

def get_corpus(file_name, use_tfidf = True):
    documents = []

    with open (file_name, 'r') as f:
        documents = f.readlines()

    processed_docs = [preprocess(text) for text in tqdm(documents)]

    dictionary = corpora.Dictionary(processed_docs)
    dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
    corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
    if use_tfidf:
        tfidf = models.TfidfModel(corpus)
        corpus = tfidf[corpus]
        
    return (corpus, dictionary)

def get_lda_topic_model(corpus, dictionary, num_topics = 10):
    t0 = time()
    #lda_model = models.LdaMulticore(corpus, num_topics=num_topics, id2word=dictionary) #Bad!
    lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary,  eval_every=5, alpha='auto', gamma_threshold=0.01)
    #lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary,  eval_every=5, iterations = 1000, alpha='auto', gamma_threshold=0.01)
    #lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary,  eval_every=5, alpha='auto') #Good enough
    #lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary)

    print("done in %fs" % (time() - t0))
    
    return lda_model

def get_lsi_topic_model(corpus, dictionary, num_topics = 10):
    t0 = time()
    lsi_model = models.LsiModel(corpus, num_topics = num_topics, id2word = dictionary)
    print("done in %fs" % (time() - t0))
    
    return lsi_model

files = ['rest_review_sample_100000', 'categories/Chinese_pos', 'categories/Chinese_neg','categories/Chinese', 'categories/Mexican']
(corpus, dictionary) = get_corpus(files[0] + ".txt")
model_1 = get_lda_topic_model(corpus, dictionary, num_topics=20)
#model_1_1 = get_lsi_topic_model(corpus, dictionary, num_topics=20)
largest_coherence = -1e20
for k in range(5, 30):
    model = get_lda_topic_model(corpus, dictionary, num_topics=k)    
    cm = models.coherencemodel.CoherenceModel(model=model, corpus=corpus, coherence='u_mass')
    coherence = cm.get_coherence()
    print("k=%d coherence=%f"%(k, coherence))
    if (coherence > largest_coherence):
        largest_coherence = coherence
        model_1 = model
    
corpus, dictionary = get_corpus(files[1] + ".txt")
model_2 = get_lda_topic_model(corpus, dictionary)

corpus, dictionary = get_corpus(files[2] + ".txt")
model_3 = get_lda_topic_model(corpus, dictionary)
# model_4 = get_topic_model(files[3] + ".txt")
# model_5 = get_topic_model(files[4] + ".txt")

100%|██████████| 100000/100000 [02:18<00:00, 721.53it/s]


done in 54.596851s
done in 53.589476s
k=5 coherence=-8.711588
done in 55.356229s
k=6 coherence=-6.006455
done in 53.863223s
k=7 coherence=-6.830794
done in 53.929561s
k=8 coherence=-9.040357
done in 52.581625s
k=9 coherence=-9.111941
done in 53.819305s
k=10 coherence=-7.393406
done in 52.411267s
k=11 coherence=-9.348815
done in 52.211808s
k=12 coherence=-9.748305
done in 53.122221s
k=13 coherence=-9.353274
done in 120.260599s
k=14 coherence=-8.486139
done in 56.933982s
k=15 coherence=-8.616813
done in 53.562596s
k=16 coherence=-9.407344
done in 52.509920s
k=17 coherence=-8.954476
done in 53.434786s
k=18 coherence=-8.718664
done in 54.582902s
k=19 coherence=-8.635861
done in 55.931944s
k=20 coherence=-8.205251
done in 54.137989s
k=21 coherence=-9.358789
done in 56.326857s
k=22 coherence=-8.500321
done in 54.698021s
k=23 coherence=-8.155904
done in 55.101050s
k=24 coherence=-8.248734
done in 53.858708s
k=25 coherence=-8.718934
done in 53.723265s
k=26 coherence=-8.272773
done in 55.207964

  0%|          | 118/27804 [00:00<00:23, 1173.14it/s]

k=29 coherence=-9.057965


  1%|          | 71/10912 [00:00<00:15, 679.88it/s]

done in 16.402430s


100%|██████████| 10912/10912 [00:14<00:00, 764.19it/s]


done in 7.212865s


In [30]:
corpus, dictionary = get_corpus(files[0] + ".txt")

100%|██████████| 100000/100000 [02:19<00:00, 718.40it/s]


In [38]:
dictionary.id2token[corpus[0]]

KeyError: 1

In [31]:
for i in corpus[0]:
    dictionary.id2token(i[0])

TypeError: 'dict' object is not callable

In [4]:
Stock_Colors = ['black', 'maroon', 'red', 'purple', 'fuchsia', 'green', 'lime', 'olive', 'navy', 'blue', 'teal', 'aqua',
               'black', 'maroon', 'red', 'purple', 'fuchsia', 'green', 'lime', 'olive']

def get_topic_json(model, title, compare_words):
    topic = model.show_topics(-1, formatted=False)
    children_name = 'children'
    name_name = 'name'
    value_name = 'value'
    color_name = 'color'    

    topic_out = {name_name: title, children_name:[]}

    for i in range(0, len(topic)):
        topic_out[children_name].append({name_name: 'Topic ' + str(i), children_name:[]})
        max_weight = topic[i][1][0][1]
        for j in range(0, len(topic[i][1])):
            topic_out[children_name][i][children_name].append({name_name:topic[i][1][j][0],
                                                               value_name:"{0:.2f}".format(topic[i][1][j][1]/max_weight),
                                                               color_name: Stock_Colors[i],
                                                               'new_word': not (topic[i][1][j][0] in compare_words)})
        
    return topic_out


def get_topic_words(model):
    topic = model.show_topics(-1, formatted=False)
    all_words = {}
    for i in range(0, len(topic)):
        for j in range(0, len(topic[i][1])):
            if topic[i][1][j][0] in all_words:
                all_words[topic[i][1][j][0]] += 1
            else:
                all_words[topic[i][1][j][0]] = 1
    
    return all_words
    
# with open(files[0] + '.json', 'w') as f:
#     f.write(json.dumps(get_topic_json(model_1, '100000_Samples', {})))

# pos_words = get_topic_words(model_2)
# neg_words = get_topic_words(model_3)

# topic2 = get_topic_json(model_2, 'Positive', neg_words)
# topic3 = get_topic_json(model_3, 'Negative', pos_words)
# topic = {'name':'Chinese Restaurant Reviews', 'children':[topic2, topic3]}
# with open('compare' + '.json', 'w') as f:
#     f.write(json.dumps(topic))
    
# topic4 = get_topic_json(model_4, 'Chinese')
# topic5 = get_topic_json(model_5, 'Mexican')
# topic = {'name':'Chinese vs. Mexican', 'children':[topic2, topic3]}
# with open('compare_cn_mx' + '.json', 'w') as f:
#     f.write(json.dumps(topic))

In [5]:
for idx, topic in model_1.print_topics(-1):
    print('Topic: {} Words: {}'.format(idx, topic))
print('-------------------------------------------------------------------')

# for idx, topic in model_1_1.print_topics(-1):
#     print('Topic: {} Words: {}'.format(idx, topic))
# print('-------------------------------------------------------------------')

for idx, topic in model_2.print_topics(-1):
    print('Topic: {} Words: {}'.format(idx, topic))
print('-------------------------------------------------------------------')

for idx, topic in model_3.print_topics(-1):
    print('Topic: {} Words: {}'.format(idx, topic))
print('-------------------------------------------------------------------')
# for idx, topic in model_4.print_topics(-1):
#     print('Topic: {} Words: {}'.format(idx, topic))
# print('-------------------------------------------------------------------')
# for idx, topic in model_5.print_topics(-1):
#     print('Topic: {} Words: {}'.format(idx, topic))

Topic: 0 Words: 0.006*"great" + 0.006*"burger" + 0.006*"fri" + 0.006*"amaz" + 0.006*"delici" + 0.005*"love" + 0.005*"chicken" + 0.005*"order" + 0.005*"good" + 0.005*"salad"
Topic: 1 Words: 0.051*"pizza" + 0.011*"crust" + 0.008*"wing" + 0.006*"minut" + 0.005*"manag" + 0.005*"pepperoni" + 0.005*"phone" + 0.004*"wait" + 0.004*"oven" + 0.003*"boba"
Topic: 2 Words: 0.007*"place" + 0.006*"taco" + 0.005*"great" + 0.005*"good" + 0.005*"time" + 0.005*"drink" + 0.004*"friend" + 0.004*"like" + 0.004*"servic" + 0.004*"beer"
Topic: 3 Words: 0.013*"cheesesteak" + 0.010*"brat" + 0.008*"flavour" + 0.008*"marrow" + 0.008*"gorgeous" + 0.008*"macaroon" + 0.007*"biz_photo" + 0.007*"firehous" + 0.006*"church" + 0.006*"termin"
Topic: 4 Words: 0.030*"pancak" + 0.028*"breakfast" + 0.025*"egg" + 0.017*"crepe" + 0.012*"hash" + 0.010*"biscuit" + 0.009*"omelet" + 0.009*"benedict" + 0.009*"coffe" + 0.009*"waffl"
Topic: 5 Words: 0.050*"sushi" + 0.023*"ayc" + 0.020*"indian" + 0.017*"roll" + 0.015*"sashimi" + 0.014*"

In [6]:
# (corpus, dictionary) = get_corpus(files[0] + ".txt")

# for k in range(5, 20):
#     model = get_lda_topic_model(corpus, dictionary, num_topics=k)    
#     cm = models.coherencemodel.CoherenceModel(model=model, corpus=corpus, coherence='u_mass')
#     print("k=%d coherence=%f"%(k,cm.get_coherence()))

In [7]:
# (corpus, dictionary) = get_corpus(files[1] + ".txt")
# hdp_2 = models.HdpModel(corpus, dictionary)
# l=hdp_2.suggested_lda_model()
# l.show_topics(20)

# (corpus, dictionary) = get_corpus(files[2] + ".txt")
# hdp_3 = models.HdpModel(corpus, dictionary)
# l=hdp_3.suggested_lda_model()
# l.show_topics(20)

http://qpleple.com/topic-coherence-to-evaluate-topic-models/

Select number of topics for LDA model: https://cran.r-project.org/web/packages/ldatuning/vignettes/topics.html